In [ ]:
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q8_0.gguf --local-dir . --local-dir-use-symlinks False

In [10]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
# using https://llama-cpp-python.readthedocs.io/en/latest/
# install package with nvidia gpu acceleratior, {cu122} version of cuda in google collab
!pip install llama-cpp-python \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122

In [ ]:
# # Run this to clean GPU memory
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
from llama_cpp import Llama

model_pathname = "./mistral-7b-instruct-v0.2.Q8_0.gguf"
# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(
  model_path=model_pathname,  # Download the model file first
  n_ctx=32768,          # 32768 # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=8,          # The number of CPU threads to use, tailor to your system and the resulting performance
  n_gpu_layers=-1,       # The number of layers to offload to GPU, if you have GPU acceleration available
  seed=-1,
)

prompt = "Majki bitz"

# output = llm(
#   f'''[INST] {prompt} [/INST]''',
#   temperature=0.3,
#   top_p=0.95,
#   top_k=40,
#   max_tokens=256,
#   stop=["</s>"],
#   echo=True
# )
# print(output)

In [76]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

chat = [
     {"role": "user", "content": "Vytvoř stručný český nadpis, který výstižně shrnuje obsah tohoto abstraktu: \n 'abstract': 'Tržby maloobchodu se v srpnu meziročně zvedly o 5,9 procenta. To představuje nejvyšší růst od listopadu roku 2004. Nejvíce se na něm podílel prodej textilu a obuvi. Uvedl to v úterý Český statistický úřad (ČSÚ).'"},
]

test_prompt = tokenizer.apply_chat_template(chat, tokenize=False)
print(test_prompt)

<s>[INST] Vytvoř stručný český nadpis, který výstižně shrnuje obsah tohoto abstraktu: 
 'abstract': 'Tržby maloobchodu se v srpnu meziročně zvedly o 5,9 procenta. To představuje nejvyšší růst od listopadu roku 2004. Nejvíce se na něm podílel prodej textilu a obuvi. Uvedl to v úterý Český statistický úřad (ČSÚ).' [/INST]


In [ ]:
output = llm(
  test_prompt,
  temperature=0.5,
  top_p=0.95,
  top_k=40,
  max_tokens=50,
  stop=["</s>"],
  echo=False
)
print(output)

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
import os

# for sumeczech dataset
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/sumeczech'

GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['sumeczech-1.0-dev.jsonl', 'sumeczech-1.0-oodtest.jsonl', 'sumeczech-1.0-test.jsonl', 'sumeczech-1.0-test.jsonl.example', 'sumeczech-1.0-oodtest.jsonl.example', 'sumeczech-1.0-dev.jsonl.example', 'sumeczech_3shot_abstract-to-headline.json']


In [9]:
import json
from typing import List, Dict
import glob
from torch.utils.data import Dataset, DataLoader

SUMECZECH_TEST = "sumeczech-1.0-test.jsonl"

class SummaryDatasetSumeCzech(Dataset):
    def __init__(self, data_path: str):
        """
        Args:
            data_path (str): Path to the data file
        """
        self.data = []
        with open(data_path, "r") as f:
            for line in f:
                self.data.append(json.loads(line))


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

dataset = SummaryDatasetSumeCzech(os.path.join(GOOGLE_DRIVE_PATH, SUMECZECH_TEST))
data_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=1)

In [ ]:
# 3 shots of prompt are from dev dataset
chat_abstract_to_headline_3shot = [
  {"role": "user", "content": "Jste užitečným pomocníkem, který shrne text v českém jazyce pro různé typy sumarizací.\n 'abstract': 'Dlouho očekávaný model Moto X, jehož vzhled si určí sám uživatel, přestavila společnost Motorola ještě pod hlavičkou Googlu již v srpnu 2013. Právě široká možnost personalizace však vadí šéfdesignérovi Applu, který si neodpustil kritiku. Motorola reagovala kritikou předražených produktů Applu.'"},
  {"role": "assistant", "content": "iPhony za nehorázné ceny nemají budoucnost, vrací Motorola kritiku Applu."},
  {"role": "user", "content": "'abstract': 'Silný vítr, který přechází s frontou přes Česko od západu, se kolem poledne opřel do hřebenů Krkonoš silou orkánu. Na Luční boudě naměřili v nárazech 127 kilometrů za hodinu. Pak vichr zeslábl, ale v noci se čeká další úder. V horách silně sněží a silnice kloužou. Horská služba nedoporučuje hřebenové túry.'"},
  {"role": "assistant", "content": "Orkán zastavil lanovku na Sněžku, hřebenové túry jsou nebezpečné."},
  {"role": "user", "content": "'abstract': 'Asi osm kilometrů severně od Turnova nad obcí Frýdštejn je 657 metrů vysoký zalesněný vrch Kopanina s 18 metrů vysokou cihlovou rozhlednou. Věž zde byla postavena Klubem českých turistů v roce 1894. Nejlépe se k ní dostaneme, když zaparkujeme auto ve stejnojmenné osadě u Myslivecké chaty Kopanina. Odtud je to asi půl kilometru pěšky po modré až k rozhledně.'"},
  {"role": "assistant", "content": "Co lze vidět na Kopanině"},
  {"role": "user", "content": ""},
]

chat_text_to_abstract_3shot = [
  {"role": "user", "content": "Jste užitečným pomocníkem, který shrne text v českém jazyce pro různé typy sumarizací. Shrň text ze zpravodajského článku do abstraktu dlouhého přibližně 3 až 6 vět\n 'text': 'Klíč od rozhledny je v domku č. 14 u autobusové zastávky. Nejbližší železniční zastávka je v Rychnově u Jablonce nad Nisou, která je vzdálena od Kopaniny přibližně čtyři kilometry.\nVystoupíme-li po 93 schodech na věž, naskytne se nám pěkný rozhled do kraje. Severními směry je vidět Ještěd s televizním vysílačem, Jizerské hory s horou Jizerou a některými rozhlednami, jako je Královka, Bramberk a Černá studnice. Severovýchodně a východně spatříme Krkonoše s horou Szrenicou s horskou chatou a televizní věž nad Sněžnými jámami v Polsku, dále Kotel, Luční horu, Žalý s rozhlednou a Černou horu s televizním vysílačem. Jihovýchodně vidíme vrchy Zvičinu, Tábor s rozhlednou, Kozákov s rozhlednou a v údolí nedaleký hrad Frýdštejn, jižně a severozápadně hrad Ralsko, Klíč v Lužických horách a další kopce. Nejbližší občerstvení je k dispozici v Myslivecké chatě Kopanina, u které jsme nechali stát auto.\nV okolí Kopaniny je pak řada dalších turisticky zajímavých míst. Tak například asi dva kilometry po modré značce se dostaneme ke zřícenině hradu Frýdštejn ze 14. století. Nedaleko od hradu můžeme zaparkovat i auta. Dominantou Frýdštejna je válcová věž, ostatní části tvoří zbytky zdí a ve skalách vytesané místnosti. K jiné zajímavé zřícenině, hradu Vranov, můžeme pokračovat zhruba další dva kilometry po červené. Na mohutném skalním bloku se z něj zachovaly malé zbytky zdí, místnosti vytesané ve skalách, kaple z roku 1826 a několik pěkných vyhlídek. V nejbližším okolí jsou také poměrně méně známé skalní útvary Suché a Klokočské skály.\nZ rozhledny na vrchu Kopanina je za jakéhokoli počasí dobře vidět nejbližší město Turnov. Jeho dominantou je novogotický kostel s nedostavěnou věží. (23. srpna 2001)'"},
  {"role": "assistant", "content": "Asi osm kilometrů severně od Turnova nad obcí Frýdštejn je 657 metrů vysoký zalesněný vrch Kopanina s 18 metrů vysokou cihlovou rozhlednou. Věž zde byla postavena Klubem českých turistů v roce 1894. Nejlépe se k ní dostaneme, když zaparkujeme auto ve stejnojmenné osadě u Myslivecké chaty Kopanina. Odtud je to asi půl kilometru pěšky po modré až k rozhledně."},
  {"role": "user", "content": "'text': 'Od rána napadlo deset až patnáct centimetrů sněhu, na Luční boudě je ho kolem šedesáti centimetrů. Vítr tam teď fouká od jihu, v nárazech kolem 90 kilometrů za hodinu,\" řekl iDNES.cz po 14. hodině Adolf Klepš, šéf Horské služby Krkonoše. Podle královéhradeckého meteorologa Jiřího Jakubského se večer a v noci čeká opět zesílení větru až na sílu orkánu, tedy nad 118 kilometrů za hodinu. Slábnutí meteorologové předpovídají až na sobotu. V nižších polohách hor zdaleka zdaleka tolik nefoukalo, například v lyžařských střediscích nebo v krkonošských městech. Všude ale silně sněžilo a klouzaly silnice. \"Problémy začínají mít řidiči v okolí Vrchlabí, Pece pod Sněžkou, Špindlerova Mlýna, ale i na silnicích v nižších polohách, například mezi obcemi Mladé Buky a Rudník ve směru od Trutnova na Vrchlabí,\" uvedl dvě hodiny po poledni policejní mluvčí Udo Ertner. Silný vítr také zastavil provoz lanovky z Růžové hory na Sněžku. Dojet bylo možné je z Pece pod Sněžkou do mezistanice, ale i o to by v pátek minimální zájem. \"Od rána přišlo asi deset lidí, nahoře je mlha,\" řekla odpoledne agentuře ČTK pracovnice lanovky. Horská služba podle Klepše nedoporučuje hřebenové túry. Pokud bude vývoj počasí pokračovat podle meteorologů, nebude je doporučovat ani o víkendu. \"Očekáváme zvýšení stupně lavinového nebezpečí nejméně na druhý stupeň, záleží na množství sněhu a síle větru,\" upřesnil Klepš. Podle meteorologů hrozí v noci a v sobotu dopoledne už v polohách nad 600 metrů nad mořem tvoření sněhových jazyků, na horách i závějí. (více o vydané výstraze čtěte zde).'"},
  {"role": "assistant", "content": "Silný vítr, který přechází s frontou přes Česko od západu, se kolem poledne opřel do hřebenů Krkonoš silou orkánu. Na Luční boudě naměřili v nárazech 127 kilometrů za hodinu. Pak vichr zeslábl, ale v noci se čeká další úder. V horách silně sněží a silnice kloužou. Horská služba nedoporučuje hřebenové túry."},
  {"role": "user", "content": "'text': 'Myšlenku dát spotřebitelům volnou ruku nad finálním vzhledem telefonu zkritizoval hlavní designér Applu Jonathan Ive v nedávném rozhovoru pro magazín New Yorker. I přes to, že výslovně požádal, aby nebyla společnost jmenována, je patrné, že kritika padá na hlavu Motoroly. Ta jediná totiž umožňuje koncovým uživatelům takovou volnost v dosažení individuálního vzhledu konečného produktu.Zobrazit fotogalerii\"Vytvořte si, co chcete. Můžete si vybrat barvu, jakou chcete,\" citoval magazín Iveovu narážku na myšlenku konkurence. Právě tento krok totiž Ive vnímá tak, že se designer Motoroly pouze zbavil odpovědnosti za konečný vzhled smartphonu. Ten nechává čistě v rukou každého uživatele.Oproti Applu, jehož dva iPhony aktuální generace jsou k dispozici pouze ve třech barevných odstínech, tak mají výrazně širší možnosti. Personifikace Moto X spočívá především ve výběru pojetí zadního krytu. Může být kožený, dřevěný, či z měkčeného plastu. A vždy v několika odstínech. Barevnému přizpůsobení pak podléhá například i taková drobnost jako mřížka sluchátka. Záda telefonu mohou být opatřena i třeba krátkým mottem či věnováním.Motorola si samozřejmě Iveovu kritika nenechala líbit. \"Naše společnost měla jinou filozofii,\" reagoval výkonný ředitel Motoroly Rick Osterloh prostřednictvím BBC. \"Naší myšlenkou je, že by koncový uživatel měl být přímo zapojen do procesu návrhu produktu,\" dodal s tím, že produktové řady Motoroly jsou dostupné širokému spektru uživatelů. Neodpustil si totiž kritiku na adresu Applu, a to kvůli pobuřující výši prodejních cen jeho produktů.Osterloh věří, že firemní strategie, za níž je společnost šéfdesignérem Applu kritizována, je správná. \"Jsme přesvědčeni, že budoucnost je v nabídce podobných možností a ve velkém výběru za dostupné ceny,\" dodal.Naopak v produktech za \"nehorázné ceny\" žádnou budoucnost nevidí. Podle Osterloha by neměl být smartphone a mobilní internet drahým luxusem, nýbrž jednoduchou volbou pro každého.Jonathan Ive je asi nejvlivnějším designérem světa, který má lví podíl na nevídaném úspěchu společnosti Apple. Jednoduchost, jakési nevyřčené motto firmy, vtiskl úspěšnému iPodu, iPhonu i tabletu iPad.\nDo Applu, s nímž již dříve spolupracoval ve vlastním poradenském studiu Tangerine, nastoupil v roce 1992. V roce 1996, rok před návratem Steva Jobse, se stal šéfem designového oddělení společnosti.\nIveovy výrobky získaly obrovskou řadu ocenění včetně prestižního Design Award Excellence v USA. Je také držitelem ocenění designéra roku od Londýnského muzea designu. Za přínos britskému impériu byl oceněn královskou rodinou nejvyšším britským řádem a povýšen do šlechtického stavu.\nBez něj by iPhone nevznikl, ale nikdo ho nezná. Sir Jonathan Ive.'"},
  {"role": "assistant", "content": "Dlouho očekávaný model Moto X, jehož vzhled si určí sám uživatel, přestavila společnost Motorola ještě pod hlavičkou Googlu již v srpnu 2013. Právě široká možnost personalizace však vadí šéfdesignérovi Applu, který si neodpustil kritiku. Motorola reagovala kritikou předražených produktů Applu."},
  {"role": "user", "content": ""},
]

chat_abstract_to_headline_0shot = [
     {"role": "user", "content": ""},
]

# Baseline will be calculated out of 150 sampels from test dataset
# Creating json data formated for benchmarking
json_data_3shot_abstract_to_headline = {
    "dataset": "sumeczech",
    "type": "3shot_abstract_to_headline",
    "model": "mistral-7b-instruct-v0.2",
    "abstracts" : [],
    "references": [],
    "predictions": []
}

json_data_3shot_text_to_abstract = {
    "dataset": "sumeczech",
    "type": "3shot_text_to_abstract",
    "model": "mistral-7b-instruct-v0.2",
    "texts" : [],
    "references": [],
    "predictions": []
}

json_data_0shot_abstract_to_headline = {
    "dataset": "sumeczech",
    "type": "0shot_abstract_to_headline",
    "model": "mistral-7b-instruct-v0.2",
    "abstracts" : [],
    "references": [],
    "predictions": []
}

for i, batch in enumerate(data_loader):
    # get abstract and headline

    abstract = batch['abstract'][0]
    headline = batch['headline'][0]
    text = batch['text'][0]

    format_content_abstract = "'abstract': '{abstract}'"
    format_content_text = "'text': '{text}'"
    format_content_abstract_0shot = "Vytvoř stručný český nadpis, který výstižně shrnuje obsah tohoto abstraktu: \n '{abstract}'"

    # create prompt
    # chat_abstract_to_headline_3shot[-1]['content'] = format_content_abstract.format(abstract=abstract)
    # chat_text_to_abstract_3shot[-1]['content'] = format_content_text.format(text=text)
    chat_abstract_to_headline_0shot[-1]['content'] = format_content_abstract_0shot.format(abstract=abstract)

    # prompt_3shot_abstract_to_headline = tokenizer.apply_chat_template(chat_abstract_to_headline_3shot, tokenize=False)
    # prompt_3shot_text_to_abstract = tokenizer.apply_chat_template(chat_text_to_abstract_3shot, tokenize=False)
    prompt_0shot_abstract_to_headline = tokenizer.apply_chat_template(chat_abstract_to_headline_0shot, tokenize=False)

    output = llm(
        prompt_0shot_abstract_to_headline,
        temperature=0.6,
        top_p=0.95,
        top_k=40,
        max_tokens=42, # 50 for abstract to headline
        stop=["</s>"],
        echo=False
    )

    # get output text
    prediction = output["choices"][0]["text"]

    # save json abstracts and their references + predictions
    # json_data_3shot_abstract_to_headline["abstracts"].append(abstract)
    # json_data_3shot_abstract_to_headline["references"].append(headline)
    # json_data_3shot_abstract_to_headline["predictions"].append(prediction)

    # json_data_3shot_text_to_abstract["texts"].append(text)
    # json_data_3shot_text_to_abstract["references"].append(abstract)
    # json_data_3shot_text_to_abstract["predictions"].append(prediction)

    json_data_0shot_abstract_to_headline["abstracts"].append(abstract)
    json_data_0shot_abstract_to_headline["references"].append(headline)
    json_data_0shot_abstract_to_headline["predictions"].append(prediction)

    if i >= 150:
        break



In [ ]:
print(json.dumps(json_data_0shot_abstract_to_headline, indent=4, ensure_ascii=False))

In [92]:

# # save json to gdrive formatted for benchmarking
# with open(os.path.join(GOOGLE_DRIVE_PATH, "sumeczech_3shot_abstract-to-headline.json"), "w") as f:
#     json.dump(json_data_3shot_abstract_to_headline, f, indent=4, ensure_ascii=False)

# save json to gdrive formatted for benchmarking
# with open(os.path.join(GOOGLE_DRIVE_PATH, "sumeczech_3shot_text-to-abstract.json"), "w") as f:
#     json.dump(json_data_3shot_text_to_abstract, f, indent=4, ensure_ascii=False)

# save json to gdrive formatted for benchmarking
with open(os.path.join(GOOGLE_DRIVE_PATH, "sumeczech_0shot_abstract-to-headline.json"), "w") as f:
    json.dump(json_data_0shot_abstract_to_headline, f, indent=4, ensure_ascii=False)